In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import linspace
from math import sqrt
from random import sample
from random import seed
import matplotlib.pyplot as plt
from numpy import corrcoef
import pickle
from numpy import isnan
%matplotlib notebook
import seaborn as sns
from numpy import median

In [2]:
# drugsInfo  = './Screened_Compounds.csv'
# drugsInfo  = pd.read_csv(drugsInfo)

drugsData = './target.csv'
drugsData = pd.read_csv(drugsData, header = 0)

pairs = './IC50.csv'
pairs = pd.read_csv(pairs)

SNP  = './SNP_CNV.csv'
SNP  = pd.read_csv(SNP)

Progeny = './progeny11.csv'
Progeny= pd.read_csv(Progeny, header = 0)

In [3]:
SNP.rename(columns={'Unnamed: 0': 'Cell line ID'},inplace=True)
Progeny.rename(columns={'Unnamed: 0': 'Cell line ID'},inplace=True)
drugsData.rename(columns={'Unnamed: 0': 'Name'},inplace=True)

In [5]:
# drugsInfo= drugsInfo.loc[:,["Drug ID","Drug Name","Target Pathway"]]
# drugsData= pd.merge(drugsData , drugsInfo,left_on = "Name", right_on = "Drug Name")
# drugsData = drugsData.loc[:, drugsData.columns != "Name"]
# drugsData.reset_index(drop=True, inplace=True)

In [21]:
drugsData


,Unnamed: 0,EGFR,MTORC1,PDGFR,KIT,VEGFR,FLT3,RET,CSF1R,MET,...,ESR1,EGLN1,RNA helicase A,Induces reactive oxygen species,NAMPT,TRAIL receptor agonist,L3MBTL3,DOT1L,DNA alkylating agent,NAE
0,Erlotinib,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rapamycin,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sunitinib,0,0,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,PHA-665752,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,MG-132,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,AZD6244,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261,Bicalutamide,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,RDEA119 (rescreen),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,GDC0941 (rescreen),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
new_pairs = {}
new_pairs['drug']=[]
new_pairs['Cell line ID']=[]
new_pairs['IC50']=[]
'''repair pairs'''
cell_lines = pairs.iloc[0].index[1:].values
for i in range(len(pairs)):
    row = pairs.iloc[i]
    drug = row['Unnamed: 0']
    for cl in cell_lines:
        cell_line_ID = int(cl)
        IC_50 = row[cl]
        if pd.isnull(IC_50) == False:
            new_pairs['drug'].append(drug)
            new_pairs['Cell line ID'].append(cell_line_ID)
            new_pairs['IC50'].append(IC_50)

        

In [5]:
pairs = pd.DataFrame.from_dict(new_pairs)

In [6]:
'''First setup: seperate model for each drug - only cell line data- Progeny'''
all_data = pd.merge(pairs, Progeny, left_on = 'Cell line ID', right_on = 'Cell line ID')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50"], axis =1)
cell_line_features.columns

Index(['EGFR', 'Hypoxia', 'JAK.STAT', 'MAPK', 'NFkB', 'PI3K', 'TGFb', 'TNFa',
       'Trail', 'VEGF', 'p53'],
      dtype='object')

In [7]:
seed(7)
random_states = sample(range(100), 5)
random_states

[41, 19, 50, 83, 6]

In [30]:
all_data

,drug,Cell line ID,IC50,EGFR,Hypoxia,JAK.STAT,MAPK,NFkB,PI3K,TGFb,TNFa,Trail,VEGF,p53
0,Erlotinib,906794,0.290898,-0.970208,1.251282,0.538888,-1.849428,-0.369537,-0.104723,1.309720,-0.112547,-0.435198,1.384981,2.308153
1,Rapamycin,906794,0.207925,-0.970208,1.251282,0.538888,-1.849428,-0.369537,-0.104723,1.309720,-0.112547,-0.435198,1.384981,2.308153
2,Sunitinib,906794,4.126633,-0.970208,1.251282,0.538888,-1.849428,-0.369537,-0.104723,1.309720,-0.112547,-0.435198,1.384981,2.308153
3,PHA-665752,906794,2.874543,-0.970208,1.251282,0.538888,-1.849428,-0.369537,-0.104723,1.309720,-0.112547,-0.435198,1.384981,2.308153
4,MG-132,906794,-1.696581,-0.970208,1.251282,0.538888,-1.849428,-0.369537,-0.104723,1.309720,-0.112547,-0.435198,1.384981,2.308153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212769,AZD6244,1322212,2.199008,-0.864721,-0.080292,-0.913660,-0.849333,-1.196522,1.453247,0.489443,-1.305121,0.156704,-1.329742,-0.698295
212770,Bicalutamide,1322212,5.811887,-0.864721,-0.080292,-0.913660,-0.849333,-1.196522,1.453247,0.489443,-1.305121,0.156704,-1.329742,-0.698295
212771,RDEA119 (rescreen),1322212,5.276282,-0.864721,-0.080292,-0.913660,-0.849333,-1.196522,1.453247,0.489443,-1.305121,0.156704,-1.329742,-0.698295
212772,GDC0941 (rescreen),1322212,1.310022,-0.864721,-0.080292,-0.913660,-0.849333,-1.196522,1.453247,0.489443,-1.305121,0.156704,-1.329742,-0.698295


In [32]:
results = {}
drugs = all_data.drug.unique()


params = {}

num = 1
for drug in drugs:
    dicti = {}
    sample = all_data[all_data['drug']== drug]
    cell_line_features = sample.drop(['drug','Cell line ID',"IC50"], axis =1)

    X = cell_line_features
    Y = sample['IC50']

    sum_RMSE_test = 0
    sum_RMSE_train = 0
    sum_corr_test = 0
    sum_corr_train = 0

    for random_state in random_states :

        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= random_state)

        # Hyperparamter tuning on trainig set
        # Instantiate the model
        elastic_net = ElasticNet(random_state= 2, tol = 1)
        # Instanitiate scaler
        scaler = StandardScaler()
        minimum = min(y_train)
        maksimum = max(y_train)
        y_train = [(x-minimum)/(maksimum-minimum) for x in y_train]
        y_test = [(x-minimum)/(maksimum-minimum) for x in y_test]
        # Initiate pipeline
        pipeline = Pipeline([
                ("scaler", scaler),
                ("estimator", elastic_net)
                ])
        param_grid = {"estimator__alpha": [0.001, 0.01, 0.1, 1.],
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
        # Instatiate grid search object
        grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)
        grid.fit(X_train, y_train)
        #second param_grid ( only if alpha !=1)
        alpha = grid.best_params_['estimator__alpha']
        if alpha != 1:
            param_grid = {"estimator__alpha": list(linspace(alpha, 10*alpha,4)),
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
            grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)


        # Train the grid (compute cross validation for every parameter combination)
        grid.fit(X_train, y_train)

        d = {}
        d['params'] = grid.best_params_
        print('dict', d['params'])
        d['cv_results'] = grid.cv_results_
        d['var_imp'] =  grid.best_estimator_['estimator'].coef_

        dicti[random_state] = d

        best_model_test_predictions = grid.predict(X_test)
        best_model_train_predictions = grid.predict(X_train)

        #y_test
        RMSE_test = sqrt(mean_squared_error(y_test, best_model_test_predictions))

        #s = {'y_test' : list(y_test), 'prediction': best_model_test_predictions}
        #s = pd.DataFrame(s)
        corr_test = corrcoef(y_test, best_model_test_predictions)[0][1]
        if isnan(corr_test)==True:
            print('1' ,drug, random_state, corr_test)
            corr_test = 0
            print('3' ,drug, random_state, corr_test)

        #y_train
        RMSE_train = sqrt(mean_squared_error(y_train, best_model_train_predictions))
        #p = {'y_train': list(y_train), 'prediction' : best_model_train_predictions}
        #p = pd.DataFrame(p)
        corr_train = corrcoef(y_train , best_model_train_predictions)[0][1]
        if isnan(corr_train) == True:
            print('2',drug, random_state, corr_train)
            corr_train = 0
            print('4' ,drug, random_state, corr_train)

        sum_RMSE_test += RMSE_test
        sum_RMSE_train += RMSE_train
        sum_corr_train += corr_train
        sum_corr_test += corr_test

        print(random_state, grid.best_params_,  corr_train, corr_test)

    RMSE_test = sum_RMSE_test/5
    RMSE_train = sum_RMSE_train/5
    corr_test = sum_corr_test/5
    corr_train = sum_corr_train/5
    results[drug] = [drug, RMSE_train, RMSE_test, corr_train, corr_test]

    print(drug , num, '!!!!')

    num +=1
    params[drug] = dicti
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])



dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
41 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 7.026892222511512e-16 7.156187510589631e-16
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
19 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} -2.3136287453445946e-16 -1.9280088181466458e-16
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
50 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.2714098628335896 0.0882898921884509
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.18957004179989775 0.13566512931896973
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
6 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.24530799316920285 0.15864987002251876
Erlotinib 1 !!!!
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
41 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.2956925633631388 0.26095188409746695
dict {'estimator__alpha': 1.0, 'est

/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
50 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.22444626734846918 0.10496656817977924
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0}
83 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0} 0.15975744197904082 0.10799761931477003
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
6 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.2241416859113483 0.10957593457599012
PHA-665752 4 !!!!
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
2 MG-132 41 nan
4 MG-132 41 0
41 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 0 2.7527064491568074e-16


/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
1 MG-132 19 nan
3 MG-132 19 0
19 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 1.0804399226950939e-16 0


...

dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
19 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.20363104087858516 0.2483267947018599
dict {'estimator__alpha': 0.001, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.001, 'estimator__l1_ratio': 0.25} 0.2241225562630546 0.20688525954535805
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.2666778945043338 -0.021172638339430658
dict {'estimator__alpha': 0.001, 'estimator__l1_ratio': 0.75}
6 {'estimator__alpha': 0.001, 'estimator__l1_ratio': 0.75} 0.26324213417187703 0.1361388059234586
TW 37 224 !!!!
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.29449784148378205 0.24877269558445342
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
19 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.29973427889172655 0.20807320513504943
dict {'estimator__alpha': 0.

dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
50 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.25743666948468585 0.22132178828994303
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.0}
83 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.0} 0.27783589720287505 0.1702778034163511
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
6 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.250745906562381 0.25707267044258386
PFI-1 234 !!!!
dict {'estimator__alpha': 0.001, 'estimator__l1_ratio': 1.0}
41 {'estimator__alpha': 0.001, 'estimator__l1_ratio': 1.0} 0.2599526119173448 0.11450160594166142
dict {'estimator__alpha': 0.4, 'estimator__l1_ratio': 0.0}
19 {'estimator__alpha': 0.4, 'estimator__l1_ratio': 0.0} 0.22695947135268194 0.20165619296492543
dict {'estimator__alpha': 0.4, 'estimator__l1_ratio': 0.0}
50 {'estimator__alpha': 0.4, 'estimator__l1_ratio': 0.0} 0.252945133397151 0.07451458789494145
dict {'estimator__alpha': 0.1, 'estimator__l1_ra

/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.1913560845297313 0.20242639440081628
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.21337112109351827 0.09821634559647195
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
6 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.17580295369541332 0.15944030990563882
BMN-673 240 !!!!
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.2041129731148962 0.184963561910017
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
19 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.19499604227259218 0.23255239735598532
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.19829438837578356 0.2053936216469744
dict {'estimator__alpha': 0.01, 'esti

dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
83 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.37253469738700673 0.33335046229523313
dict {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0}
6 {'estimator__alpha': 1.0, 'estimator__l1_ratio': 0.0} 0.3448123068821181 0.40959790327040285
Bleomycin (50 uM) 250 !!!!
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.26881768862651206 0.20105686021948652
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
19 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.2601574954188763 0.18609820100679966
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.0}
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.0} 0.2757309469746303 0.21088395873114263
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.2800814140806335 0.23439903726410874
dict {'estimator__alpha': 0.01, 'e

In [33]:
results.describe()

,RMSE_train,RMSE_test,corr_train,corr_test
count,256.000000,256.000000,256.000000,256.000000
mean,0.152338,0.154558,0.336548,0.287908
std,0.030081,0.030540,0.116253,0.122262
min,0.079645,0.078336,0.062374,-0.024382
25%,0.130853,0.135630,0.243391,0.195663
50%,0.154228,0.156811,0.332507,0.286128
75%,0.173642,0.175872,0.426527,0.381176
max,0.240507,0.246644,0.605938,0.612203


In [34]:
results.to_csv('./prediction_per_drug_progeny_EN.csv')

In [35]:
with open('params_per_drug_progeny_EN.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
'''First setup: seperate model for each cell line - only drug data- SNP'''
all_data = pd.merge(pairs, SNP, left_on = 'Cell line ID', right_on = 'Cell line ID')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50"], axis =1)
cell_line_features.columns


Index(['ABCB1_mut', 'ABL2_mut', 'ACACA_mut', 'ACVR1B_mut', 'ACVR2A_mut',
       'ADCY1_mut', 'AFF4_mut', 'AHCTF1_mut', 'AHNAK_mut', 'AKAP9_mut',
       ...
       'loss:cnaPANCAN416', 'loss:cnaPANCAN417', 'loss:cnaPANCAN418',
       'loss:cnaPANCAN419', 'loss:cnaPANCAN420', 'loss:cnaPANCAN421',
       'loss:cnaPANCAN422', 'loss:cnaPANCAN423', 'loss:cnaPANCAN424',
       'loss:cnaPANCAN425'],
      dtype='object', length=735)

In [39]:
results = {}
drugs = all_data.drug.unique()


params = {}

num = 1
for drug in drugs:
    dicti = {}
    sample = all_data[all_data['drug']== drug]
    cell_line_features = sample.drop(['drug','Cell line ID',"IC50"], axis =1)

    X = cell_line_features
    Y = sample['IC50']

    sum_RMSE_test = 0
    sum_RMSE_train = 0
    sum_corr_test = 0
    sum_corr_train = 0

    for random_state in random_states :

        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= random_state)

        # Hyperparamter tuning on trainig set
        # Instantiate the model
        elastic_net = ElasticNet(random_state= 2, tol = 1)
        # Instanitiate scaler
        scaler = StandardScaler()
        minimum = min(y_train)
        maksimum = max(y_train)
        y_train = [(x-minimum)/(maksimum-minimum) for x in y_train]
        y_test = [(x-minimum)/(maksimum-minimum) for x in y_test]
        # Initiate pipeline
        pipeline = Pipeline([
                ("scaler", scaler),
                ("estimator", elastic_net)
                ])
        param_grid = {"estimator__alpha": [0.001, 0.01, 0.1, 1.],
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
        # Instatiate grid search object
        grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)
        grid.fit(X_train, y_train)
        #second param_grid ( only if alpha !=1)
        alpha = grid.best_params_['estimator__alpha']
        if alpha != 1:
            param_grid = {"estimator__alpha": list(linspace(alpha, 10*alpha,4)),
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
            grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)


        # Train the grid (compute cross validation for every parameter combination)
        grid.fit(X_train, y_train)

        d = {}
        d['params'] = grid.best_params_
        print('dict', d['params'])
        d['cv_results'] = grid.cv_results_
        d['var_imp'] =  grid.best_estimator_['estimator'].coef_

        dicti[random_state] = d

        best_model_test_predictions = grid.predict(X_test)
        best_model_train_predictions = grid.predict(X_train)

        #y_test
        RMSE_test = sqrt(mean_squared_error(y_test, best_model_test_predictions))

        #s = {'y_test' : list(y_test), 'prediction': best_model_test_predictions}
        #s = pd.DataFrame(s)
        corr_test = corrcoef(y_test, best_model_test_predictions)[0][1]
        if isnan(corr_test)==True:
            print('1' ,drug, random_state, corr_test)
            corr_test = 0
            print('3' ,drug, random_state, corr_test)

        #y_train
        RMSE_train = sqrt(mean_squared_error(y_train, best_model_train_predictions))
        #p = {'y_train': list(y_train), 'prediction' : best_model_train_predictions}
        #p = pd.DataFrame(p)
        corr_train = corrcoef(y_train , best_model_train_predictions)[0][1]
        if isnan(corr_train) == True:
            print('2',drug, random_state, corr_train)
            corr_train = 0
            print('4' ,drug, random_state, corr_train)

        sum_RMSE_test += RMSE_test
        sum_RMSE_train += RMSE_train
        sum_corr_train += corr_train
        sum_corr_test += corr_test

        print(random_state, grid.best_params_,  corr_train, corr_test)

    RMSE_test = sum_RMSE_test/5
    RMSE_train = sum_RMSE_train/5
    corr_test = sum_corr_test/5
    corr_train = sum_corr_train/5
    results[drug] = [drug, RMSE_train, RMSE_test, corr_train, corr_test]

    print(drug , num, '!!!!')

    num +=1
    params[drug] = dicti
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])

dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
41 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 7.026892222511512e-16 7.156187510589631e-16
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
19 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} -2.3136287453445946e-16 -1.9280088181466458e-16
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.4191331770299525 0.010487514209688029
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.3888148761019608 0.03094283118213662
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.39423746729437065 -0.07033433066582892
Erlotinib 1 !!!!
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
41 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 2.3304996356718e-16 1.743416963806218e-16
dict {'estimator__alpha': 0.

/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.75}
2 Rapamycin 50 nan
4 Rapamycin 50 0
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.75} 0 -1.7910201870072363e-16


/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
2 Rapamycin 83 nan
4 Rapamycin 83 0
83 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 0 4.1932690796530366e-17


...

/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
19 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} -3.1163686978975446e-16 1.0192529217068499e-15
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} 3.150329673758709e-16 -1.9134667175169567e-16
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.4031221792594511 0.0033336568239411093
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.3938146017998362 0.05149668337175847
XAV 939 244 !!!!
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75} 0.5816540435576619 0.6401627205663438
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0}
19 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0} 0.5591357004281688 0.5049580597406151
dict {'estimator__alpha

dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5}
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.5} -7.6870359574297e-17 2.924241545943186e-16
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.15152937351591853 0.1131801545134236
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.1669637586075498 0.06970580423221043
GDC0941 (rescreen) 254 !!!!
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
41 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.24399617238848678 0.017091438316031254
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
19 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.2873984092790269 0.19647714688193543
dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.3120896411849662 0.1348856014456539
dict {'estimator__alpha': 0.

/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ewa/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


dict {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.1, 'estimator__l1_ratio': 0.25} 0.2268283907306778 0.08490539121743762
MLN4924 255 !!!!
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
41 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.4481803628344722 0.05301705814880816
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
19 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.3866449816062439 0.09168335621725811
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.46150319739676515 0.017965292047281978
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.39470279334456065 0.11691340309503785
dict {'estimator__alpha': 0.07, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.07, 'estimator__l1_ratio': 0.25} 0.2759804726459452 -0.002354573989151967
PF-4708671 256 !!!!


In [40]:
results.describe()

,RMSE_train,RMSE_test,corr_train,corr_test
count,256.000000,256.000000,2.560000e+02,256.000000
mean,0.151594,0.159734,3.398423e-01,0.142276
std,0.031133,0.031336,2.046699e-01,0.134726
min,0.080550,0.081723,-2.877347e-16,-0.027929
25%,0.130250,0.137281,1.651608e-01,0.014418
50%,0.151507,0.160637,3.809575e-01,0.121966
75%,0.170214,0.180115,5.077814e-01,0.224388
max,0.250331,0.256161,6.936697e-01,0.652055


In [41]:
results.to_csv('./prediction_per_drug_SNP_EN.csv')

In [42]:
with open('params_per_drug_SNP_EN.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
'''Second setup - per cell line- only drug data'''
all_data = pd.merge(pairs, drugsData, left_on = 'drug', right_on = 'Name')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50","Name",], axis =1)
cell_line_features.columns


Index(['EGFR', 'MTORC1', 'PDGFR', 'KIT', 'VEGFR', 'FLT3', 'RET', 'CSF1R',
       'MET', 'Proteasome',
       ...
       'ESR1', 'EGLN1', 'RNA helicase A', 'Induces reactive oxygen species',
       'NAMPT', 'TRAIL receptor agonist', 'L3MBTL3', 'DOT1L',
       'DNA alkylating agent', 'NAE'],
      dtype='object', length=228)

In [8]:
all_data

,drug,Cell line ID,IC50,Name,EGFR,MTORC1,PDGFR,KIT,VEGFR,FLT3,...,ESR1,EGLN1,RNA helicase A,Induces reactive oxygen species,NAMPT,TRAIL receptor agonist,L3MBTL3,DOT1L,DNA alkylating agent,NAE
0,Erlotinib,906794,0.290898,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Erlotinib,753531,-1.863485,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Erlotinib,753532,3.974344,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Erlotinib,753583,1.385394,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Erlotinib,910911,3.857275,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227996,MLN4924,1298160,-0.240773,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227997,MLN4924,908120,1.616776,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227998,MLN4924,905943,0.130265,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227999,MLN4924,1298144,0.497722,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
results = {}
cells = all_data['Cell line ID'].unique()


params = {}

num = 1
for cl in cells:
    dicti = {}
    sample = all_data[all_data['Cell line ID']== cl]
    cell_line_features = sample.drop(['drug','Cell line ID',"IC50","Name"], axis =1)

    X = cell_line_features
    Y = sample['IC50']

    sum_RMSE_test = 0
    sum_RMSE_train = 0
    sum_corr_test = 0
    sum_corr_train = 0

    for random_state in random_states :

        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= random_state)

        # Hyperparamter tuning on trainig set
        # Instantiate the model
        elastic_net = ElasticNet(random_state= 2, tol = 1)
        # Instanitiate scaler
        scaler = StandardScaler()
        minimum = min(y_train)
        maksimum = max(y_train)
        y_train = [(x-minimum)/(maksimum-minimum) for x in y_train]
        y_test = [(x-minimum)/(maksimum-minimum) for x in y_test]
        # Initiate pipeline
        pipeline = Pipeline([
                ("scaler", scaler),
                ("estimator", elastic_net)
                ])
        param_grid = {"estimator__alpha": [0.001, 0.01, 0.1, 1.],
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
        # Instatiate grid search object
        grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)
        grid.fit(X_train, y_train)
        #second param_grid ( only if alpha !=1)
        alpha = grid.best_params_['estimator__alpha']
        if alpha != 1:
            param_grid = {"estimator__alpha": list(linspace(alpha, 10*alpha,4)),
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}
            grid = GridSearchCV(pipeline, param_grid,
                        scoring="neg_mean_squared_error",
                        cv=5,
                        refit=True)


        # Train the grid (compute cross validation for every parameter combination)
        grid.fit(X_train, y_train)

        d = {}
        d['params'] = grid.best_params_
        print('dict', d['params'])
        d['cv_results'] = grid.cv_results_
        d['var_imp'] =  grid.best_estimator_['estimator'].coef_

        dicti[random_state] = d

        best_model_test_predictions = grid.predict(X_test)
        best_model_train_predictions = grid.predict(X_train)

        #y_test
        RMSE_test = sqrt(mean_squared_error(y_test, best_model_test_predictions))

        #s = {'y_test' : list(y_test), 'prediction': best_model_test_predictions}
        #s = pd.DataFrame(s)
        corr_test = corrcoef(y_test, best_model_test_predictions)[0][1]
        if isnan(corr_test)==True:
            print('1' ,drug, random_state, corr_test)
            corr_test = 0
            print('3' ,drug, random_state, corr_test)

        #y_train
        RMSE_train = sqrt(mean_squared_error(y_train, best_model_train_predictions))
        #p = {'y_train': list(y_train), 'prediction' : best_model_train_predictions}
        #p = pd.DataFrame(p)
        corr_train = corrcoef(y_train , best_model_train_predictions)[0][1]
        if isnan(corr_train) == True:
            print('2',drug, random_state, corr_train)
            corr_train = 0
            print('4' ,drug, random_state, corr_train)

        sum_RMSE_test += RMSE_test
        sum_RMSE_train += RMSE_train
        sum_corr_train += corr_train
        sum_corr_test += corr_test

        print(random_state, grid.best_params_,  corr_train, corr_test)

    RMSE_test = sum_RMSE_test/5
    RMSE_train = sum_RMSE_train/5
    corr_test = sum_corr_test/5
    corr_train = sum_corr_train/5
    results[cl] = [cl, RMSE_train, RMSE_test, corr_train, corr_test]

    print(cl , num, '!!!!')

    num +=1
    params[cl] = dicti
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])


dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75} 0.8665442769866905 0.5750541750927933
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
19 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.8399244696930342 0.4747723828496506
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
50 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.8928287894897015 0.399112371377628
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5}
83 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.5} 0.915588537837944 0.41512440282397056
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.25} 0.928421129665943 0.48618438055213326
906794 1 !!!!
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75}
41 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75} 0.9162645063039747 0.42299579144306754
dict {'estimator__alpha': 0.01, 'estimator_

/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
6 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.8930164039199215 0.5830624475333696
753531 2 !!!!
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
41 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.7984372837020781 0.6107449342446504
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0}
19 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 1.0} 0.8262079943478645 0.3613930639147717
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
50 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.8177248636756532 0.4250159082969515
dict {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25}
83 {'estimator__alpha': 0.04, 'estimator__l1_ratio': 0.25} 0.7552055738263252 0.5162151684417147
dict {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75}
6 {'estimator__alpha': 0.01, 'estimator__l1_ratio': 0.75} 0.8370302936858749 0.4490185713093814
753532 3 !!!!
dict {'estimator__alpha': 0.

KeyboardInterrupt: 

In [71]:
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])


In [72]:
results.describe()

,name,RMSE_train,RMSE_test,corr_train,corr_test
count,9.640000e+02,964.000000,964.000000,964.000000,964.000000
mean,9.908448e+05,0.108484,0.167030,0.832553,0.394034
std,2.200043e+05,0.017749,0.022302,0.100626,0.115866
min,6.836650e+05,0.037181,0.110712,0.121697,-0.054623
25%,9.068110e+05,0.096880,0.153434,0.820854,0.340726
50%,9.097225e+05,0.106154,0.165748,0.857645,0.411854
75%,1.240143e+06,0.116798,0.178031,0.884467,0.470044
max,1.789883e+06,0.202370,0.353142,0.994826,0.621228


In [73]:
results.to_csv('./prediction_per_cell_EN.csv')

In [74]:
with open('params_per_cell_EN.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
'''Third setup'''
all_data = pd.merge(pairs, drugsData, left_on = 'drug', right_on = 'Name')
all_data=pd.merge(all_data, Progeny, left_on = 'Cell line ID', right_on = 'Cell line ID')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50","Name"], axis =1)
drugs = all_data.drug.unique()
cell_line_features.columns

Index(['EGFR_x', 'MTORC1', 'PDGFR', 'KIT', 'VEGFR', 'FLT3', 'RET', 'CSF1R',
       'MET', 'Proteasome',
       ...
       'Hypoxia', 'JAK.STAT', 'MAPK', 'NFkB', 'PI3K', 'TGFb', 'TNFa', 'Trail',
       'VEGF', 'p53'],
      dtype='object', length=239)

In [9]:
results = []
sets = []

params = {}


for random_state in random_states:
    dicti = {}
    
    cells_in_pairs = all_data["Cell line ID"].unique() #921 unique cell lines
    
    seed(random_state)
    Validation_set = sample(set(cells_in_pairs), k =130)
    cells_in_pairs = [x  for x in cells_in_pairs if x not in Validation_set]
    Validation_set = all_data[all_data['Cell line ID'].isin(Validation_set)]
    
    seed(random_state)
    Test_set = sample(set(cells_in_pairs), k= 131)
    Train_set = [x for x in cells_in_pairs if x not in Test_set]
    Test_set = all_data[all_data['Cell line ID'].isin(Test_set)]
    Train_set = all_data[all_data['Cell line ID'].isin(Train_set)]

    y_val = Validation_set['IC50']
    y_test = Test_set['IC50']
    y_train = Train_set['IC50']
    
    minimum = min(y_train)
    maximum = max(y_train)
    
    y_train = [(x-minimum)/(maximum - minimum) for x in y_train]
    y_test = [(x-minimum)/(maximum-minimum) for x in y_test]
    y_val = [(x-minimum)/(maximum-minimum) for x in y_val]

    X_val = Validation_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)
    X_test = Test_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)
    X_train = Train_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)
    
    Train_set.loc[:,'IC50'] = y_train
    Test_set.loc[:,'IC50'] = y_test
    Validation_set.loc[:,'IC50'] = y_val
    
    print(min(y_train),max(y_train),min(y_test),max(y_test),min(y_val),max(y_val))

    scaler = StandardScaler()

    param_grid = {"estimator__alpha": [0.001, 0.01, 0.1, 1.],
                    "estimator__l1_ratio": [0., 0.25, 0.5, 0.75, 1.]}

    parameters = ParameterGrid(param_grid)
    alpha = 0
    l1_ratio = 0
    RMSE = 100

    for param in parameters:
        #initiate the model
        elastic_net = ElasticNet(random_state=2, 
                                 alpha = param["estimator__alpha"], 
                                 l1_ratio = param[ "estimator__l1_ratio"])
        #initiate pipeline
        pipeline = Pipeline([
            ("scaler", scaler),
            ("estimator", elastic_net)
            ])
        pipeline.fit(X_train,y_train)
        y_pred = pipeline.predict(X_val)
        #print('alpha:' ,param["estimator__alpha"], 'l1_ratio:',  param[ "estimator__l1_ratio"],
             #"RMSE_val:" , sqrt(mean_squared_error(y_val, y_pred)))
        if sqrt(mean_squared_error(y_val, y_pred)) < RMSE:
            RMSE = sqrt(mean_squared_error(y_val, y_pred))
            alpha = param["estimator__alpha"]
            l1_ratio = param[ "estimator__l1_ratio"]

    #initiate the model
    elastic_net = ElasticNet(random_state=2,
                             alpha = alpha, 
                             l1_ratio = l1_ratio)
    #Initiate pipeline
    pipeline = Pipeline([
           ("scaler", scaler),
           ("estimator", elastic_net)
            ])

    #new train set containing Train_set and Validation_set
    Train_set = pd.concat([Train_set, Validation_set])
    y_train = Train_set['IC50']
    X_train = Train_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)

    #fit pipeline
    pipeline.fit(X_train,y_train)

    #prediction
    y_pred_test = pipeline.predict(X_test)
    y_pred_train = pipeline.predict(X_train)
    
    Train_set['IC50_pred'] = y_pred_train
    Test_set['IC50_pred'] = y_pred_test
    drugs = all_data.drug.unique()
    
    sets.append((Train_set, Test_set))
    
    dicti['params'] = {'alpha': alpha, 'l1_ratio': l1_ratio}
    
    dicti['var_imp'] =  pipeline['estimator'].coef_
    
    params[random_state] = dicti
    
    predictions = {}
    for drug in drugs:
        train_sample = Train_set[Train_set['Name']== drug]
        test_sample = Test_set[Test_set['Name']== drug]
    
        RMSE_test = sqrt(mean_squared_error(test_sample['IC50'], test_sample["IC50_pred"]))
        RMSE_train = sqrt(mean_squared_error(train_sample['IC50'], train_sample["IC50_pred"]))

        corr_test =  corrcoef(test_sample['IC50'], test_sample["IC50_pred"])[0][1]
        if isnan(corr_test) == True:
            corr_test = 0
        
        corr_train =  corrcoef(train_sample['IC50'], train_sample["IC50_pred"])[0][1]
        if isnan(corr_train) == True:
            corr_train = 0
        predictions[drug] = [drug, RMSE_test,RMSE_train , corr_test, corr_train , random_state]
    results.append(predictions)
    
    print(random_state, "!")

/home/ewa/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0.0 1.0 0.006566463194809993 0.9664323233787868 0.04384011105352056 1.0310847243734957


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 425.1107440291681, tolerance: 0.24035285892148697
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 435.8546686822656, tolerance: 0.24035285892148697
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 507.3963846435355, tolerance: 0.24035285892148697
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of

41 !


/home/ewa/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


...

83 !


/home/ewa/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0.0 1.0 -0.006409317865408688 0.9517430163497113 0.04802147257162958 0.9696591797810092


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 402.16647305207346, tolerance: 0.22798340586441104
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 412.5267798954424, tolerance: 0.22798340586441104
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 480.892922623999, tolerance: 0.22798340586441104
  positive)
/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of

6 !


In [10]:
for i in range(len(sets)):
    sets[i] = pd.concat([sets[i][0].assign(dataset = 'Train_set'),sets[i][1].assign(dataset = 'Test_set')]).sort_index()

In [12]:
with open('./params_all_progeny_EN.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
all_drugs_model = {}
for drug in drugs:
    RMSE_test = 0
    RMSE_train = 0
    corr_test  = 0
    corr_train = 0
    for i in range(len(results)):
        RMSE_test += results[i][drug][1]
        RMSE_train += results[i][drug][2]
        corr_test += results[i][drug][3]
        corr_train += results[i][drug][4]
    all_drugs_model[drug] = [drug, RMSE_test/5, RMSE_train/5, corr_test/5, corr_train/5]
results_all_drugs = pd.DataFrame.from_dict(all_drugs_model, orient='index',
                            columns=['name','RMSE_test', 'RMSE_train', 'corr_test', 'corr_train'])


In [12]:
results_all_drugs.to_csv('./prediction_all_progeny_EN.csv')

In [13]:
results_all_drugs.describe()

,RMSE_test,RMSE_train,corr_test,corr_train
count,256.000000,256.000000,256.000000,256.000000
mean,0.067804,0.067129,0.222678,0.244057
std,0.026149,0.025925,0.140205,0.147324
min,0.023373,0.022872,-0.246638,-0.260526
25%,0.048509,0.048466,0.130261,0.148636
50%,0.063875,0.063479,0.229418,0.249087
75%,0.080211,0.079624,0.335569,0.360363
max,0.174764,0.178617,0.488892,0.518069
